In [5]:
# import Pkg
# Pkg.add("HTTP")
# Pkg.add("JSON")
# Pkg.add("YAML")
# Pkg.add("URIs")

In [649]:
include("../src/QuantumAnnealers.jl")
using .QuantumAnnealers
using JSON, HTTP, URIs

In [650]:
using YAML

# Load your Dwave API key into config
config = YAML.load_file("../secrets.yaml");
QuantumAnnealers.init(config["dwave"]["key"]);

In [651]:
# # Define your Ising problem
# h = Dict(1 => -1, 2 => -1)  # Linear coefficients
# J = Dict((1, 2) => 1)     # Quadratic coefficients

# # Submit the Ising problem to the solver
# job_id = QuantumAnnealers.ising_sample(solver["topology__type"], h, J)

# # Assuming the job is complete, fetch the results
# results = QuantumAnnealers.fetch_results(job_id)

# println(results)

In [652]:
QuantumAnnealers.solvers_available()

hybrid_binary_quadratic_model_version2
	 Status: ONLINE	 Avg Load: 0.0 
hybrid_discrete_quadratic_model_version1
	 Status: ONLINE	 Avg Load: 0.0 
Advantage_system4.1
	 Status: ONLINE	 Avg Load: 0.0 
hybrid_constrained_quadratic_model_version1
	 Status: ONLINE	 Avg Load: 0.0 
Advantage2_prototype1.1
	 Status: ONLINE	 Avg Load: 0.0 
Advantage_system6.3
	 Status: ONLINE	 Avg Load: 0.0 


6-element Vector{Any}:
 Dict{String, Any}("status" => "ONLINE", "properties" => Dict{String, Any}("maximum_number_of_biases" => 200000000, "parameters" => Dict{String, Any}("time_limit" => "Maximum requested runtime in seconds."), "quota_conversion_rate" => 20, "maximum_time_limit_hrs" => 24.0, "minimum_time_limit" => Any[Any[1, 3.0], Any[1024, 3.0], Any[4096, 10.0], Any[10000, 40.0], Any[30000, 200.0], Any[100000, 600.0], Any[1000000, 600.0]], "maximum_number_of_variables" => 1000000, "supported_problem_types" => Any["bqm"], "category" => "hybrid", "version" => "2.2"), "id" => "hybrid_binary_quadratic_model_version2", "description" => "Hybrid solver for general BQM problems", "avg_load" => 0.0)
 Dict{String, Any}("status" => "ONLINE", "properties" => Dict{String, Any}("maximum_number_of_biases" => 5000000000, "parameters" => Dict{String, Any}("time_limit" => "Maximum requested runtime in seconds."), "quota_conversion_rate" => 20, "maximum_number_of_cases" => 500000, "maximum_time_limi

In [653]:
dwave_model = QuantumAnnealers.get_solver()

┌ Info: Getting solver Advantage_system6.3: ONLINE
└ @ Main.QuantumAnnealers /Users/javier/Desktop/Julia/QuantumAnnealers.jl/src/QuantumAnnealers.jl:30


Dict{String, Any} with 31 entries:
  "problem_run_duration_range"         => Any[0.0, 1.0e6]
  "programming_thermalization_range"   => Any[0.0, 10000.0]
  "readout_thermalization_range"       => Any[0.0, 10000.0]
  "couplers"                           => Any[Any[30, 31], Any[31, 32], Any[32,…
  "supported_problem_types"            => Any["ising", "qubo"]
  "j_range"                            => Any[-1.0, 1.0]
  "qubits"                             => Any[30, 31, 32, 33, 34, 35, 36, 37, 3…
  "problem_timing_data"                => Dict{String, Any}("reverse_annealing_…
  "anneal_offset_ranges"               => Any[Any[0.0, 0.0], Any[0.0, 0.0], Any…
  "parameters"                         => Dict{String, Any}("anneal_schedule"=>…
  "chip_id"                            => "Advantage_system6.3"
  "quota_conversion_rate"              => 1
  "id"                                 => "Advantage_system6.3"
  "h_range"                            => Any[-4.0, 4.0]
  "default_programming_thermaliza

In [654]:
#Ising Sampling https://docs.dwavesys.com/docs/latest/rest_api_full_examples.html#submit-a-problem-to-a-qpu-sampler

biases = Dict("x"=> -0.5, "y"=> 0.5, "xy"=> -1.0)
# biases = Dict("x"=> 4.0, "y"=> 4.0, "xy"=> -1)
# qubits = res["qubits"]
# couplers = res["couplers"]
# xy = couplers[1]

lin, quad = QuantumAnnealers.parse_b_and_j(biases, dwave_model)

conditions = QuantumAnnealers.create_conditions(lin, quad, dwave_model; label="Test 1", num_reads=10)

res = QuantumAnnealers.post_problem(conditions)

answer = QuantumAnnealers.get_answer(res)

Dict{String, Any} with 7 entries:
  "timing"           => Dict{String, Any}("qpu_delay_time_per_sample"=>20.54, "…
  "num_variables"    => 5760
  "format"           => "qp"
  "active_variables" => "HgAAAB8AAAA="
  "energies"         => "AAAAAAAA8L8AAAAAAADwvw=="
  "num_occurrences"  => "BAAAAAYAAAA="
  "solutions"        => "AMA="

In [645]:
# answer = QuantumAnnealers.get_answer(res)

Dict{String, Any} with 7 entries:
  "timing"           => Dict{String, Any}("qpu_delay_time_per_sample"=>20.54, "…
  "num_variables"    => 5760
  "format"           => "qp"
  "active_variables" => "HgAAAB8AAAA="
  "energies"         => "AAAAAAAA8L8AAAAAAADwvw=="
  "num_occurrences"  => "AgAAAAgAAAA="
  "solutions"        => "AMA="

In [655]:
@show decode_energy(answer["energies"])
@show bytes_to_ising_solutions(answer["solutions"])
@show decode_variables(answer["active_variables"])

decode_energy(answer["energies"]) = [-1.0, -1.0]
bytes_to_ising_solutions(answer["solutions"]) = [-1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1]
decode_variables(answer["active_variables"]) = Int32[30, 31]


2-element reinterpret(Int32, ::Vector{UInt8}):
 30
 31

In [656]:
get_spins(answer)

2×2 Matrix{Int64}:
 1  -1
 1  -1